In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [3]:
# Generating Empty Aggregator to be loaded 

setting = 'FedEM'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1
    
# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 262.70it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:30<00:00,  2.66it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.195% |Test Loss: 2.292 | Test Acc: 12.291% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [4]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 
    
dataloader = Custom_Dataloader(data_x, data_y)

In [35]:
# Import Model Weights
num_models = 40

adv_mode = False
Dverge_mode = False

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if adv_mode:
    if Dverge_mode:
        args_.save_path = 'weights/cifar10/DVERGE/fedEM_dverge/'
        weights = np.load('weights/cifar10/DVERGE/fedEM_dverge/train_client_weights.npy')
    else:
        args_.save_path = 'weights/final/cifar/fig3/fedem_defend/'
        weights = np.load('weights/final/cifar/fig3/fedem_defend/train_client_weights.npy')
else:
    args_.save_path = 'weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/'
    weights = np.load('weights/cifar/21_12_30_feddef_n40_linf0_5_G0_0/train_client_weights.npy')
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]

# Set model weights
model_weights = [(1,0,0), (0,1,0), (0,0,1),
                 (0,0.5,0.5), (0.5,0,0.5), (0.5,0.5,0),
                 (0.33,0.33,0.34)]

# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

In [36]:
# Here we will make a dictionary that will hold results
logs_adv = []

for i in range(num_models):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]
    

In [37]:
# Perform transfer attack from one client to another and record stats

# Run Measurements for both targetted and untargeted analysis
new_num_models = len(models_test)
victim_idxs = range(new_num_models)
custom_batch_size = 500
eps = 4.5


# Record number of batch sizes
batch_size_recs = np.ones([new_num_models,new_num_models])

for adv_idx in victim_idxs:
    print("\t Adv idx:", adv_idx)
    
    # dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
    batch_size_recs[adv_idx,:] *= batch_size
    
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    t1.generate_victims(victim_idxs)
    
    # Perform Attacks Targeted
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    
    
    
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)

    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

    # Miss attack Untargeted
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6


In [38]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([new_num_models, new_num_models]) 
orig_sim = np.zeros([new_num_models, new_num_models]) 
adv_acc = np.zeros([new_num_models, new_num_models]) 
adv_sim_target = np.zeros([new_num_models, new_num_models]) 
adv_sim_untarget = np.zeros([new_num_models, new_num_models]) 
adv_target = np.zeros([new_num_models, new_num_models])
adv_miss = np.zeros([new_num_models, new_num_models]) 

for adv_idx in range(new_num_models):
    for victim in range(new_num_models):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [39]:
orig_acc

array([[0.72, 0.68, 0.72, 0.71, 0.74, 0.75, 0.74],
       [0.68, 0.68, 0.74, 0.71, 0.76, 0.74, 0.76],
       [0.69, 0.70, 0.76, 0.72, 0.74, 0.74, 0.74],
       [0.69, 0.67, 0.74, 0.69, 0.76, 0.72, 0.75],
       [0.66, 0.69, 0.75, 0.70, 0.74, 0.76, 0.74],
       [0.68, 0.70, 0.77, 0.70, 0.76, 0.72, 0.73],
       [0.71, 0.69, 0.74, 0.70, 0.77, 0.75, 0.76]])

In [40]:
adv_miss

array([[0.03, 0.36, 0.40, 0.35, 0.14, 0.12, 0.18],
       [0.34, 0.04, 0.33, 0.14, 0.35, 0.16, 0.19],
       [0.33, 0.33, 0.02, 0.16, 0.15, 0.35, 0.21],
       [0.24, 0.11, 0.09, 0.04, 0.10, 0.12, 0.06],
       [0.09, 0.22, 0.09, 0.10, 0.03, 0.10, 0.06],
       [0.12, 0.10, 0.27, 0.13, 0.13, 0.03, 0.08],
       [0.14, 0.14, 0.11, 0.07, 0.07, 0.08, 0.04]])

In [41]:
adv_target

array([[0.74, 0.09, 0.09, 0.04, 0.45, 0.48, 0.29],
       [0.28, 0.59, 0.09, 0.11, 0.23, 0.40, 0.24],
       [0.30, 0.06, 0.67, 0.11, 0.39, 0.19, 0.21],
       [0.32, 0.22, 0.31, 0.39, 0.43, 0.39, 0.45],
       [0.52, 0.08, 0.28, 0.10, 0.64, 0.40, 0.44],
       [0.61, 0.22, 0.09, 0.07, 0.42, 0.69, 0.43],
       [0.51, 0.15, 0.21, 0.18, 0.57, 0.55, 0.55]])